In [1]:
import numpy as np

In [3]:
import numpy as np

# 文件路径保持不变
file_path = 'digit_123.mat'

print(f"尝试使用 numpy.loadtxt 读取文件: {file_path}")

try:
    # 使用 numpy.loadtxt 来读取文本格式的数据文件
    # comments='#' 会告诉函数自动忽略所有以 '#' 开头的注释行
    data = np.loadtxt(file_path, comments='#')
    
    # 查看加载后数据的形状 (shape) 和前几行，确认是否成功
    print("✅ 成功加载数据！")
    print("数据形状 (rows, columns):", data.shape)
    print("数据前5行:\n", data[:5, :])

except Exception as e:
    print(f"读取文件时依然出错: {e}")

尝试使用 numpy.loadtxt 读取文件: digit_123.mat
读取文件时依然出错: the number of columns changed from 2 to 256 at row 2; use `usecols` to select a subset and avoid this error
